In [ ]:
import pandas as pd
import geopandas as gpd
import os
import pathlib as pl
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties

In [ ]:
cwd = os.getcwd()
charts_figs_dir = pl.Path('..','Scenario_Notebook', 'Charts_Figures').resolve()
print(cwd)
print(charts_figs_dir)

In [ ]:
one2one_stats_df = pd.read_csv(os.path.join(charts_figs_dir,'K_1to1_stats.csv'))
one2one_stats_df

In [ ]:
# Make color ramp for charts
iter0ramp = ['#FAF5CD', '#FAF3B5', '#FAF19D', '#FBEF85', '#FBED6D', '#FCEB55', '#FCE93D', '#FDE725']
iter1ramp = ['#D1F5D3', '#C0EEC2', '#B0E8B2', '#9FE2A2', '#8FDB92', '#7ED582', '#6ECF72', '#5EC962']
iter2ramp = ['#D0F6F4', '#B7E7E5', '#9ED9D6', '#85CAC7', '#6CBCB8', '#53ADA9', '#3A9F9A', '#21918C']
iter3ramp = ['#C9D7F7', '#B4C4E7', '#A0B1D8', '#8C9EC8', '#778BB9', '#6378A9', '#4F659A', '#3B528B']
iter4ramp = ['#EFCBF7','#D6AEDF','#BE91C8','#A574B1','#8D5799','#743A82','#5C1D6B','#440154']

iter0ramp.reverse()
iter1ramp.reverse()
iter2ramp.reverse()
iter3ramp.reverse()
iter4ramp.reverse()

In [ ]:
# Split out obs and pp
one2one_stats_df[['obs','pp']] = one2one_stats_df['Scenario'].str.split('_',expand=True)
one2one_stats_df['Iteration'] = one2one_stats_df['Iteration'].astype(int)
one2one_stats_df['Realizations'] = one2one_stats_df['Realizations'].astype(int)
one2one_stats_df['Group'] = one2one_stats_df['Group'].astype(int)
one2one_stats_df

In [ ]:
# RMS(m/d)
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(15,9))
plt.rcParams['font.family'] = 'Calibri'
plt.subplots_adjust(wspace=0.15, hspace=0.3)
ax = ax.flatten()

for i, ax in zip(np.arange(1,13,1), ax):
    group_df = one2one_stats_df[one2one_stats_df['Group'] == i]
    group_df.reset_index(inplace=True, drop=True)
    
    # Filter to just certain columns
    iter123_mcc = group_df[['obs','pp','Iteration','Realizations','Group','Res Mean(m/d)','RMS(m/d)','RMS(%)']]
    # Make copy of each iteration, rename to Iteration number
    iter0_mcc = iter123_mcc[iter123_mcc['Iteration']==0].copy()
    iter0_mcc = iter0_mcc.drop(columns='Iteration')
    iter0_mcc.rename(columns={'RMS(m/d)':'Iteration0'}, inplace=True)
    iter1_mcc = iter123_mcc[iter123_mcc['Iteration']==1].copy()
    iter1_mcc = iter1_mcc.drop(columns='Iteration')
    iter1_mcc.rename(columns={'RMS(m/d)':'Iteration1'}, inplace=True)
    iter2_mcc = iter123_mcc[iter123_mcc['Iteration']==2].copy()
    iter2_mcc = iter2_mcc.drop(columns='Iteration')
    iter2_mcc.rename(columns={'RMS(m/d)':'Iteration2'}, inplace=True)
    iter3_mcc = iter123_mcc[iter123_mcc['Iteration']==3].copy()
    iter3_mcc = iter3_mcc.drop(columns='Iteration')
    iter3_mcc.rename(columns={'RMS(m/d)':'Iteration3'}, inplace=True)
    iter4_mcc = iter123_mcc[iter123_mcc['Iteration']==4].copy()
    iter4_mcc = iter4_mcc.drop(columns='Iteration')
    iter4_mcc.rename(columns={'RMS(m/d)':'Iteration4'}, inplace=True)
    # Merge to 1 dataframe
    iter123_mcc = pd.merge(iter0_mcc, iter1_mcc[['Realizations','Iteration1']], on='Realizations', how='left')
    iter123_mcc = pd.merge(iter123_mcc, iter2_mcc[['Realizations','Iteration2']], on='Realizations', how='left')
    iter123_mcc = pd.merge(iter123_mcc, iter3_mcc[['Realizations','Iteration3']], on='Realizations', how='left')
    iter123_mcc = pd.merge(iter123_mcc, iter4_mcc[['Realizations','Iteration4']], on='Realizations', how='left')
    iter123_mcc = iter123_mcc.sort_values(by='Realizations')
    iter123_mcc.reset_index(inplace=True, drop=True)
    iter123_mcc['position'] = iter123_mcc.index+1

    # give colorramps to iter123_mcc
    iter123_mcc = iter123_mcc.sort_values(by='Iteration0')
    iter123_mcc['Iter0ramp'] = iter0ramp
    iter123_mcc = iter123_mcc.sort_values(by='Iteration1')
    iter123_mcc['Iter1ramp'] = iter1ramp
    iter123_mcc = iter123_mcc.sort_values(by='Iteration2')
    iter123_mcc['Iter2ramp'] = iter2ramp
    iter123_mcc = iter123_mcc.sort_values(by='Iteration3')
    iter123_mcc['Iter3ramp'] = iter3ramp
    iter123_mcc = iter123_mcc.sort_values(by='Iteration4')
    iter123_mcc['Iter4ramp'] = iter4ramp
    iter123_mcc = iter123_mcc.sort_index()

    if iter123_mcc['obs'][0] == 'R25':
        Observations = 'Random 25'
    if iter123_mcc['obs'][0] == 'R100':
        Observations = 'Random 100'
    if iter123_mcc['obs'][0] == 'C25':
        Observations = 'Clustered 25'
    if iter123_mcc['obs'][0] == 'C100':
        Observations = 'Clustered 100'
        
    if iter123_mcc['pp'][0] == 'pp10':
        PilotPoints = 800
    if iter123_mcc['pp'][0] == 'pp25':
        PilotPoints = 128
    if iter123_mcc['pp'][0] == 'pp50':
        PilotPoints = 32
      
    # ax
    plt.rcParams['font.family'] = 'Calibri'
    ax.scatter(iter123_mcc['position']-0.3,iter123_mcc['Iteration0'], color='#FDE725', edgecolors= "grey", s= 75, linewidth=1)
    ax.scatter(iter123_mcc['position']-0.15,iter123_mcc['Iteration1'], color=iter123_mcc['Iter1ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.scatter(iter123_mcc['position'],iter123_mcc['Iteration2'], color=iter123_mcc['Iter2ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.scatter(iter123_mcc['position']+0.15,iter123_mcc['Iteration3'], color=iter123_mcc['Iter3ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.scatter(iter123_mcc['position']+0.3,iter123_mcc['Iteration4'], color=iter123_mcc['Iter4ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.set_xticks([1,2,3,4,5,6,7,8],['10','25','50','100','250','500','1000','2000'], fontsize=10)
    ax.set_yticks([0, 1,2,3,4,5,6,7,8])
    #ax.set_xlabel('Number of Realizations')
    ax.set_title(f"Pilot Points: {PilotPoints}", fontsize=12)

    if group_df.at[0,'Group'] == 1:
        ax.set_ylabel("RMS(m/d)")
    if group_df.at[0,'Group'] == 5:
        ax.set_ylabel("RMS(m/d)")
    if group_df.at[0,'Group'] == 9:
        ax.set_ylabel("RMS(m/d)")

    if group_df.at[0,'Group'] == 9:
        ax.set_xlabel('Number of Realizations')
    if group_df.at[0,'Group'] == 10:
        ax.set_xlabel('Number of Realizations')
    if group_df.at[0,'Group'] == 11:
        ax.set_xlabel('Number of Realizations')
    if group_df.at[0,'Group'] == 12:
        ax.set_xlabel('Number of Realizations')
    
    # Get the x-tick locations
    xticks = ax.get_xticks()
    xticks = xticks +0.5
    xticks = xticks.tolist()
    xticks.pop()
    
    # Draw vertical lines between ticks
    for tick in xticks:
        ax.axvline(x=tick, color='gray', linestyle=(0, (1,1)), linewidth = 0.5)

#fig.suptitle('Mathews Correlation Coefficient Score with Probability >50%', fontweight='bold', fontsize=20)
plt.gcf().text(0.15, 0.91, 'Observations: Random 25', fontsize=12,fontweight='bold')
plt.gcf().text(0.35, 0.91, 'Observations: Random 100', fontsize=12, fontweight='bold')
plt.gcf().text(0.55, 0.91, 'Observations: Clustered 25', fontsize=12, fontweight='bold')
plt.gcf().text(0.75, 0.91, 'Observations: Clustered 100', fontsize=12, fontweight='bold')

# Legend Items
iter0dot = Line2D([],[], color='#FDE725', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Prior')
iter1dot = Line2D([],[], color='#5EC962', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 1')
iter2dot = Line2D([],[], color='#21918C', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 2')
iter3dot = Line2D([],[], color='#3B528B', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 3')
iter4dot = Line2D([],[], color='#440154', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 4')

LegendElement = [iter0dot, iter1dot, iter2dot, iter3dot, iter4dot]
LegendElement = [i for i in LegendElement if i is not None]

# legend
fig.legend(handles=LegendElement, title=None, title_fontsize=13, loc='lower center', frameon=False, ncol=5,
          prop= {'size':12, 'family':'calibri'})# bbox_to_anchor=(-0.05,1.4))

fig.savefig('k_RMS_md.png', bbox_inches='tight', dpi=300)

In [ ]:
# # Res Mean
# fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(15,9))
# plt.rcParams['font.family'] = 'Calibri'
# plt.subplots_adjust(wspace=0.15, hspace=0.3)
# ax = ax.flatten()

# for i, ax in zip(np.arange(1,13,1), ax):
#     group_df = one2one_stats_df[one2one_stats_df['Group'] == i]
#     group_df.reset_index(inplace=True, drop=True)
    
#     # Filter to just certain columns
#     iter123_mcc = group_df[['obs','pp','Iteration','Realizations','Group','Res Mean(m/d)','RMS(m/d)','RMS(%)']]
#     # Make copy of each iteration, rename to Iteration number
#     iter1_mcc = iter123_mcc[iter123_mcc['Iteration']==1].copy()
#     iter1_mcc = iter1_mcc.drop(columns='Iteration')
#     iter1_mcc.rename(columns={'Res Mean(m/d)':'Iteration1'}, inplace=True)
#     iter2_mcc = iter123_mcc[iter123_mcc['Iteration']==2].copy()
#     iter2_mcc = iter2_mcc.drop(columns='Iteration')
#     iter2_mcc.rename(columns={'Res Mean(m/d)':'Iteration2'}, inplace=True)
#     iter3_mcc = iter123_mcc[iter123_mcc['Iteration']==3].copy()
#     iter3_mcc = iter3_mcc.drop(columns='Iteration')
#     iter3_mcc.rename(columns={'Res Mean(m/d)':'Iteration3'}, inplace=True)
#     iter4_mcc = iter123_mcc[iter123_mcc['Iteration']==4].copy()
#     iter4_mcc = iter4_mcc.drop(columns='Iteration')
#     iter4_mcc.rename(columns={'Res Mean(m/d)':'Iteration4'}, inplace=True)
#     # Merge to 1 dataframe
#     iter123_mcc = pd.merge(iter1_mcc, iter2_mcc[['Realizations','Iteration2']], on='Realizations', how='left')
#     iter123_mcc = pd.merge(iter123_mcc, iter3_mcc[['Realizations','Iteration3']], on='Realizations', how='left')
#     iter123_mcc = pd.merge(iter123_mcc, iter4_mcc[['Realizations','Iteration4']], on='Realizations', how='left')
#     iter123_mcc = iter123_mcc.sort_values(by='Realizations')
#     iter123_mcc.reset_index(inplace=True, drop=True)
#     iter123_mcc['position'] = iter123_mcc.index+1

#     # give colorramps to iter123_mcc
#     iter123_mcc = iter123_mcc.sort_values(by='Iteration1')
#     iter123_mcc['Iter1ramp'] = iter1ramp
#     iter123_mcc = iter123_mcc.sort_values(by='Iteration2')
#     iter123_mcc['Iter2ramp'] = iter2ramp
#     iter123_mcc = iter123_mcc.sort_values(by='Iteration3')
#     iter123_mcc['Iter3ramp'] = iter3ramp
#     iter123_mcc = iter123_mcc.sort_values(by='Iteration4')
#     iter123_mcc['Iter4ramp'] = iter4ramp
#     iter123_mcc = iter123_mcc.sort_index()

#     if iter123_mcc['obs'][0] == 'R25':
#         Observations = 'Random 25'
#     if iter123_mcc['obs'][0] == 'R100':
#         Observations = 'Random 100'
#     if iter123_mcc['obs'][0] == 'C25':
#         Observations = 'Clustered 25'
#     if iter123_mcc['obs'][0] == 'C100':
#         Observations = 'Clustered 100'
        
#     if iter123_mcc['pp'][0] == 'pp10':
#         PilotPoints = 800
#     if iter123_mcc['pp'][0] == 'pp25':
#         PilotPoints = 128
#     if iter123_mcc['pp'][0] == 'pp50':
#         PilotPoints = 32
      
#     # ax
#     plt.rcParams['font.family'] = 'Calibri'
#     ax.scatter(iter123_mcc['position']-0.25,iter123_mcc['Iteration1'], color=iter123_mcc['Iter1ramp'], edgecolors= "grey", s= 75, linewidth=1)
#     ax.scatter(iter123_mcc['position']-0.1,iter123_mcc['Iteration2'], color=iter123_mcc['Iter2ramp'], edgecolors= "grey", s= 75, linewidth=1)
#     ax.scatter(iter123_mcc['position']+0.1,iter123_mcc['Iteration3'], color=iter123_mcc['Iter3ramp'], edgecolors= "grey", s= 75, linewidth=1)
#     ax.scatter(iter123_mcc['position']+0.25,iter123_mcc['Iteration4'], color=iter123_mcc['Iter4ramp'], edgecolors= "grey", s= 75, linewidth=1)
#     ax.set_xticks([1,2,3,4,5,6,7,8],['10','25','50','100','250','500','1000','2000'], fontsize=10)
#     ax.set_yticks([0, 1,2,3,4,5])
#     #ax.set_xlabel('Number of Realizations')
#     ax.set_title(f"Pilot Points: {PilotPoints}", fontsize=12)

#     if group_df.at[0,'Group'] == 1:
#         ax.set_ylabel("Res Mean(m/d)")
#     if group_df.at[0,'Group'] == 5:
#         ax.set_ylabel("Res Mean(m/d)")
#     if group_df.at[0,'Group'] == 9:
#         ax.set_ylabel("Res Mean(m/d)")

#     if group_df.at[0,'Group'] == 9:
#         ax.set_xlabel('Number of Realizations')
#     if group_df.at[0,'Group'] == 10:
#         ax.set_xlabel('Number of Realizations')
#     if group_df.at[0,'Group'] == 11:
#         ax.set_xlabel('Number of Realizations')
#     if group_df.at[0,'Group'] == 12:
#         ax.set_xlabel('Number of Realizations')
    
#     # Get the x-tick locations
#     xticks = ax.get_xticks()
#     xticks = xticks +0.5
#     xticks = xticks.tolist()
#     xticks.pop()
    
#     # Draw vertical lines between ticks
#     for tick in xticks:
#         ax.axvline(x=tick, color='gray', linestyle=(0, (1,1)), linewidth = 0.5)

# #fig.suptitle('Mathews Correlation Coefficient Score with Probability >50%', fontweight='bold', fontsize=20)
# plt.gcf().text(0.15, 0.91, 'Observations: Random 25', fontsize=12,fontweight='bold')
# plt.gcf().text(0.35, 0.91, 'Observations: Random 100', fontsize=12, fontweight='bold')
# plt.gcf().text(0.55, 0.91, 'Observations: Clustered 25', fontsize=12, fontweight='bold')
# plt.gcf().text(0.75, 0.91, 'Observations: Clustered 100', fontsize=12, fontweight='bold')

# # Legend Items
# iter1dot = Line2D([],[], color='#DEA600', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 1')
# iter2dot = Line2D([],[], color='#115C8E', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 2')
# iter3dot = Line2D([],[], color='#186A37', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 3')
# iter4dot = Line2D([],[], color='#a80084', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 4')

# LegendElement = [iter1dot, iter2dot, iter3dot,iter4dot]
# LegendElement = [i for i in LegendElement if i is not None]

# # legend
# fig.legend(handles=LegendElement, title=None, title_fontsize=13, loc='lower center', frameon=False, ncol=4,
#           prop= {'size':12, 'family':'calibri'})# bbox_to_anchor=(-0.05,1.4))

# fig.savefig('k_ResMean.png', bbox_inches='tight', dpi=300)

In [ ]:
# RMS(%)
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(15,9))
plt.rcParams['font.family'] = 'Calibri'
plt.subplots_adjust(wspace=0.15, hspace=0.3)
ax = ax.flatten()

for i, ax in zip(np.arange(1,13,1), ax):
    group_df = one2one_stats_df[one2one_stats_df['Group'] == i]
    group_df.reset_index(inplace=True, drop=True)
    
    # Filter to just certain columns
    iter123_mcc = group_df[['obs','pp','Iteration','Realizations','Group','Res Mean(m/d)','RMS(m/d)','RMS(%)']]
    # Make copy of each iteration, rename to Iteration number
    iter1_mcc = iter123_mcc[iter123_mcc['Iteration']==1].copy()
    iter1_mcc = iter1_mcc.drop(columns='Iteration')
    iter1_mcc.rename(columns={'RMS(%)':'Iteration1'}, inplace=True)
    iter2_mcc = iter123_mcc[iter123_mcc['Iteration']==2].copy()
    iter2_mcc = iter2_mcc.drop(columns='Iteration')
    iter2_mcc.rename(columns={'RMS(%)':'Iteration2'}, inplace=True)
    iter3_mcc = iter123_mcc[iter123_mcc['Iteration']==3].copy()
    iter3_mcc = iter3_mcc.drop(columns='Iteration')
    iter3_mcc.rename(columns={'RMS(%)':'Iteration3'}, inplace=True)
    iter4_mcc = iter123_mcc[iter123_mcc['Iteration']==4].copy()
    iter4_mcc = iter4_mcc.drop(columns='Iteration')
    iter4_mcc.rename(columns={'RMS(%)':'Iteration4'}, inplace=True)
    # Merge to 1 dataframe
    iter123_mcc = pd.merge(iter1_mcc, iter2_mcc[['Realizations','Iteration2']], on='Realizations', how='left')
    iter123_mcc = pd.merge(iter123_mcc, iter3_mcc[['Realizations','Iteration3']], on='Realizations', how='left')
    iter123_mcc = pd.merge(iter123_mcc, iter4_mcc[['Realizations','Iteration4']], on='Realizations', how='left')
    iter123_mcc = iter123_mcc.sort_values(by='Realizations')
    iter123_mcc.reset_index(inplace=True, drop=True)
    iter123_mcc['position'] = iter123_mcc.index+1

    # give colorramps to iter123_mcc
    iter123_mcc = iter123_mcc.sort_values(by='Iteration1')
    iter123_mcc['Iter1ramp'] = iter1ramp
    iter123_mcc = iter123_mcc.sort_values(by='Iteration2')
    iter123_mcc['Iter2ramp'] = iter2ramp
    iter123_mcc = iter123_mcc.sort_values(by='Iteration3')
    iter123_mcc['Iter3ramp'] = iter3ramp
    iter123_mcc = iter123_mcc.sort_values(by='Iteration4')
    iter123_mcc['Iter4ramp'] = iter4ramp
    iter123_mcc = iter123_mcc.sort_index()

    if iter123_mcc['obs'][0] == 'R25':
        Observations = 'Random 25'
    if iter123_mcc['obs'][0] == 'R100':
        Observations = 'Random 100'
    if iter123_mcc['obs'][0] == 'C25':
        Observations = 'Clustered 25'
    if iter123_mcc['obs'][0] == 'C100':
        Observations = 'Clustered 100'
        
    if iter123_mcc['pp'][0] == 'pp10':
        PilotPoints = 800
    if iter123_mcc['pp'][0] == 'pp25':
        PilotPoints = 128
    if iter123_mcc['pp'][0] == 'pp50':
        PilotPoints = 32
      
    # ax
    plt.rcParams['font.family'] = 'Calibri'
    ax.scatter(iter123_mcc['position']-0.25,iter123_mcc['Iteration1'], color=iter123_mcc['Iter1ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.scatter(iter123_mcc['position']-0.1,iter123_mcc['Iteration2'], color=iter123_mcc['Iter2ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.scatter(iter123_mcc['position']+0.1,iter123_mcc['Iteration3'], color=iter123_mcc['Iter3ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.scatter(iter123_mcc['position']+0.25,iter123_mcc['Iteration4'], color=iter123_mcc['Iter4ramp'], edgecolors= "grey", s= 75, linewidth=1)
    ax.set_xticks([1,2,3,4,5,6,7,8],['10','25','50','100','250','500','1000','2000'], fontsize=10)
    ax.set_yticks([0, 10,20,30,40,50,60,70,80,90])
    #ax.set_xlabel('Number of Realizations')
    ax.set_title(f"Pilot Points: {PilotPoints}", fontsize=12)

    if group_df.at[0,'Group'] == 1:
        ax.set_ylabel("RMS(%)")
    if group_df.at[0,'Group'] == 5:
        ax.set_ylabel("RMS(%)")
    if group_df.at[0,'Group'] == 9:
        ax.set_ylabel("RMS(%)")

    if group_df.at[0,'Group'] == 9:
        ax.set_xlabel('Number of Realizations')
    if group_df.at[0,'Group'] == 10:
        ax.set_xlabel('Number of Realizations')
    if group_df.at[0,'Group'] == 11:
        ax.set_xlabel('Number of Realizations')
    if group_df.at[0,'Group'] == 12:
        ax.set_xlabel('Number of Realizations')
    
    # Get the x-tick locations
    xticks = ax.get_xticks()
    xticks = xticks +0.5
    xticks = xticks.tolist()
    xticks.pop()
    
    # Draw vertical lines between ticks
    for tick in xticks:
        ax.axvline(x=tick, color='gray', linestyle=(0, (1,1)), linewidth = 0.5)

#fig.suptitle('Mathews Correlation Coefficient Score with Probability >50%', fontweight='bold', fontsize=20)
plt.gcf().text(0.15, 0.91, 'Observations: Random 25', fontsize=12,fontweight='bold')
plt.gcf().text(0.35, 0.91, 'Observations: Random 100', fontsize=12, fontweight='bold')
plt.gcf().text(0.55, 0.91, 'Observations: Clustered 25', fontsize=12, fontweight='bold')
plt.gcf().text(0.75, 0.91, 'Observations: Clustered 100', fontsize=12, fontweight='bold')

# Legend Items
iter1dot = Line2D([],[], color='#DEA600', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 1')
iter2dot = Line2D([],[], color='#115C8E', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 2')
iter3dot = Line2D([],[], color='#186A37', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 3')
iter4dot = Line2D([],[], color='#a80084', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 4')

LegendElement = [iter1dot, iter2dot, iter3dot,iter4dot]
LegendElement = [i for i in LegendElement if i is not None]

# legend
fig.legend(handles=LegendElement, title=None, title_fontsize=13, loc='lower center', frameon=False, ncol=3,
          prop= {'size':12, 'family':'calibri'})# bbox_to_anchor=(-0.05,1.4))

fig.savefig('k_RMS_percent.png', bbox_inches='tight', dpi=300)